In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 17:54:38.822537 140209855174464 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 17:54:38.829378 140209855174464 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 17:54:38.835401 140209855174464 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 17:54:38.840952 140209855174464 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 17:54:38.845859 140209855174464 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 17:54:38.854705 140209855174464 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 17:54:38.898071 140209855174464 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 17:54:38.974126 140209855174464 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9280 train_acc= 0.4357 val_loss= 1.9314 val_acc= 0.4133 time= 1.1280
Epoch: 0002 train_loss= 1.9093 train_acc= 0.4500 val_loss= 1.9157 val_acc= 0.4333 time= 0.0236
Epoch: 0003 train_loss= 1.8882 train_acc= 0.4571 val_loss= 1.8982 val_acc= 0.4400 time= 0.0288
Epoch: 0004 train_loss= 1.8657 train_acc= 0.4643 val_loss= 1.8800 val_acc= 0.4467 time= 0.0272
Epoch: 0005 train_loss= 1.8425 train_acc= 0.4643 val_loss= 1.8613 val_acc= 0.4667 time= 0.0183
Epoch: 0006 train_loss= 1.8193 train_acc= 0.4643 val_loss= 1.8429 val_acc= 0.4700 time= 0.0199
Epoch: 0007 train_loss= 1.7969 train_acc= 0.4643 val_loss= 1.8254 val_acc= 0.4733 time= 0.0201
Epoch: 0008 train_loss= 1.7754 train_acc= 0.4643 val_loss= 1.8088 val_acc= 0.4700 time= 0.0241
Epoch: 0009 train_loss= 1.7554 train_acc= 0.4643 val_loss= 1.7933 val_acc= 0.4600 time= 0.0249
Epoch: 0010 train_loss= 1.7374 train_acc= 0.4643 val_loss= 1.7797 val_acc= 0.4333 time= 0.0251
Epoch: 0011 train_loss= 1.7217 train_acc= 0.4500 v

Epoch: 0094 train_loss= 0.6395 train_acc= 0.9143 val_loss= 0.9549 val_acc= 0.7833 time= 0.0251
Epoch: 0095 train_loss= 0.6326 train_acc= 0.9143 val_loss= 0.9503 val_acc= 0.7933 time= 0.0226
Epoch: 0096 train_loss= 0.6263 train_acc= 0.9214 val_loss= 0.9463 val_acc= 0.7900 time= 0.0243
Epoch: 0097 train_loss= 0.6206 train_acc= 0.9214 val_loss= 0.9437 val_acc= 0.7933 time= 0.0193
Epoch: 0098 train_loss= 0.6151 train_acc= 0.9214 val_loss= 0.9399 val_acc= 0.7967 time= 0.0288
Epoch: 0099 train_loss= 0.6096 train_acc= 0.9214 val_loss= 0.9353 val_acc= 0.8000 time= 0.0246
Epoch: 0100 train_loss= 0.6045 train_acc= 0.9286 val_loss= 0.9305 val_acc= 0.7967 time= 0.0232
Epoch: 0101 train_loss= 0.5991 train_acc= 0.9286 val_loss= 0.9247 val_acc= 0.8000 time= 0.0231
Epoch: 0102 train_loss= 0.5937 train_acc= 0.9214 val_loss= 0.9188 val_acc= 0.8000 time= 0.0249
Epoch: 0103 train_loss= 0.5894 train_acc= 0.9214 val_loss= 0.9143 val_acc= 0.7967 time= 0.0242
Epoch: 0104 train_loss= 0.5851 train_acc= 0.9143 v

Epoch: 0186 train_loss= 0.3460 train_acc= 0.9786 val_loss= 0.7388 val_acc= 0.8200 time= 0.0242
Epoch: 0187 train_loss= 0.3457 train_acc= 0.9786 val_loss= 0.7409 val_acc= 0.8233 time= 0.0237
Epoch: 0188 train_loss= 0.3445 train_acc= 0.9786 val_loss= 0.7400 val_acc= 0.8200 time= 0.0191
Epoch: 0189 train_loss= 0.3427 train_acc= 0.9786 val_loss= 0.7377 val_acc= 0.8167 time= 0.0245
Epoch: 0190 train_loss= 0.3409 train_acc= 0.9786 val_loss= 0.7339 val_acc= 0.8200 time= 0.0280
Epoch: 0191 train_loss= 0.3399 train_acc= 0.9714 val_loss= 0.7304 val_acc= 0.8267 time= 0.0257
Epoch: 0192 train_loss= 0.3395 train_acc= 0.9714 val_loss= 0.7294 val_acc= 0.8267 time= 0.0242
Epoch: 0193 train_loss= 0.3394 train_acc= 0.9714 val_loss= 0.7291 val_acc= 0.8333 time= 0.0246
Epoch: 0194 train_loss= 0.3379 train_acc= 0.9714 val_loss= 0.7275 val_acc= 0.8400 time= 0.0202
Epoch: 0195 train_loss= 0.3348 train_acc= 0.9786 val_loss= 0.7262 val_acc= 0.8367 time= 0.0246
Epoch: 0196 train_loss= 0.3323 train_acc= 0.9786 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.7818
accuracy = 0.8110


# Gaussian Graph Convolution

In [15]:
B1 = 5e-4
B2 = 5e-4

In [16]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

W1110 17:54:44.703981 140209855174464 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:28: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 17:54:44.705257 140209855174464 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update

In [17]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1, B2), H1)

W1110 17:54:44.744564 140209855174464 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/losses.py:28: kl_divergence (from tensorflow.python.ops.distributions.kullback_leibler) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [19]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9415 train_acc= 0.3143 val_loss= 1.9418 val_acc= 0.3400 time= 0.5033
Epoch: 0002 train_loss= 1.9344 train_acc= 0.4000 val_loss= 1.9371 val_acc= 0.3867 time= 0.0238
Epoch: 0003 train_loss= 1.9290 train_acc= 0.4571 val_loss= 1.9324 val_acc= 0.4233 time= 0.0280
Epoch: 0004 train_loss= 1.9204 train_acc= 0.4714 val_loss= 1.9260 val_acc= 0.4767 time= 0.0228
Epoch: 0005 train_loss= 1.9126 train_acc= 0.5357 val_loss= 1.9202 val_acc= 0.4833 time= 0.0232
Epoch: 0006 train_loss= 1.9049 train_acc= 0.5714 val_loss= 1.9135 val_acc= 0.5100 time= 0.0237
Epoch: 0007 train_loss= 1.8963 train_acc= 0.5857 val_loss= 1.9066 val_acc= 0.4933 time= 0.0183
Epoch: 0008 train_loss= 1.8867 train_acc= 0.5929 val_loss= 1.8993 val_acc= 0.4833 time= 0.0235
Epoch: 0009 train_loss= 1.8764 train_acc= 0.6071 val_loss= 1.8919 val_acc= 0.4933 time= 0.0189
Epoch: 0010 train_loss= 1.8653 train_acc= 0.6071 val_loss= 1.8830 val_acc= 0.5167 time= 0.0276
Epoch: 0011 train_loss= 1.8566 train_acc= 0.6071 v

Epoch: 0093 train_loss= 0.7911 train_acc= 0.8857 val_loss= 1.0794 val_acc= 0.7433 time= 0.0246
Epoch: 0094 train_loss= 0.7847 train_acc= 0.8786 val_loss= 1.0744 val_acc= 0.7467 time= 0.0193
Epoch: 0095 train_loss= 0.7780 train_acc= 0.8929 val_loss= 1.0691 val_acc= 0.7567 time= 0.0186
Epoch: 0096 train_loss= 0.7714 train_acc= 0.8857 val_loss= 1.0631 val_acc= 0.7600 time= 0.0191
Epoch: 0097 train_loss= 0.7644 train_acc= 0.8857 val_loss= 1.0574 val_acc= 0.7600 time= 0.0188
Epoch: 0098 train_loss= 0.7580 train_acc= 0.8857 val_loss= 1.0522 val_acc= 0.7600 time= 0.0192
Epoch: 0099 train_loss= 0.7522 train_acc= 0.8857 val_loss= 1.0467 val_acc= 0.7633 time= 0.0202
Epoch: 0100 train_loss= 0.7470 train_acc= 0.8857 val_loss= 1.0416 val_acc= 0.7600 time= 0.0198
Epoch: 0101 train_loss= 0.7403 train_acc= 0.8857 val_loss= 1.0358 val_acc= 0.7667 time= 0.0195
Epoch: 0102 train_loss= 0.7347 train_acc= 0.8929 val_loss= 1.0312 val_acc= 0.7700 time= 0.0195
Epoch: 0103 train_loss= 0.7293 train_acc= 0.8929 v

Epoch: 0187 train_loss= 0.4361 train_acc= 0.9643 val_loss= 0.8066 val_acc= 0.8267 time= 0.0244
Epoch: 0188 train_loss= 0.4340 train_acc= 0.9643 val_loss= 0.8047 val_acc= 0.8267 time= 0.0246
Epoch: 0189 train_loss= 0.4313 train_acc= 0.9643 val_loss= 0.8037 val_acc= 0.8267 time= 0.0288
Epoch: 0190 train_loss= 0.4292 train_acc= 0.9643 val_loss= 0.8025 val_acc= 0.8300 time= 0.0230
Epoch: 0191 train_loss= 0.4273 train_acc= 0.9643 val_loss= 0.8030 val_acc= 0.8300 time= 0.0240
Epoch: 0192 train_loss= 0.4250 train_acc= 0.9643 val_loss= 0.8020 val_acc= 0.8300 time= 0.0298
Epoch: 0193 train_loss= 0.4232 train_acc= 0.9643 val_loss= 0.8009 val_acc= 0.8300 time= 0.0295
Epoch: 0194 train_loss= 0.4202 train_acc= 0.9643 val_loss= 0.7995 val_acc= 0.8300 time= 0.0287
Epoch: 0195 train_loss= 0.4185 train_acc= 0.9643 val_loss= 0.7977 val_acc= 0.8300 time= 0.0246
Epoch: 0196 train_loss= 0.4155 train_acc= 0.9643 val_loss= 0.7961 val_acc= 0.8300 time= 0.0188
Epoch: 0197 train_loss= 0.4137 train_acc= 0.9643 v

In [20]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8237
accuracy = 0.8250


# Attacks